In [66]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from pymongo import MongoClient
import json

In [2]:
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"}

In [3]:
def saveString(html, filename="test.html"):
    try:
        file = open(filename,"w")
        file.write(str(html))
        file.close()
    except Exception as ex:
        print('Error: ' + str(ex))

def loadString(f="test.html"):
    try:
        html = open(f, "r", encoding='utf-8').read()
        return(html)
    except Exception as ex:
        print('Error: ' + str(ex))

In [27]:
years = ['2018','2019','2020','2021']

prefix_url = "https://www.iplt20.com/stats/"

base_fname = 'iplstats'

In [41]:
for year in years:
    url = prefix_url + year #+ suffix_url
    page = requests.get(url, headers = headers)
    
    filename = base_fname + year + '_batting.html'
    
    saveString(page.content, filename)
    time.sleep(5)

In [88]:
player_dict = dict()
for year in years:
    
    filename = base_fname + year + '_batting.html' 
    page = loadString(filename)
    soup = BeautifulSoup(page, 'html.parser')
    
    table = soup.find("table", class_ ="np-mostruns_table")
    rows = table.find_all('tr')

    for i, row in enumerate(rows[1:]):
        stats = dict()
        data = row.find_all('td')

        if i == 0:
            player_name = data[1].find('h3').text
        else:
            player_name = data[1].find('h3', class_ ="np-team-name").text
            
        stats['matches'] = data[2].text
        stats['not_outs'] = data[4].text
        stats['high_score'] = data[5].text
        stats['avg'] = data[6].text
        stats['Average'] = data[7].text
        stats['strike_rate'] = data[9].text
        stats['num_100'] = data[10].text
        stats['num_50'] = data[11].text
        stats['num_fours'] = data[12].text
        stats['num_six'] = data[13].text
        
        year_stats = {year : stats}
        
        if player_name not in player_dict.keys():
            player_dict[player_name] = dict()
            player_dict[player_name]['Batting'] = [year_stats]
        else:
            player_dict[player_name]['Batting'].append(year_stats)

In [4]:
years = ['2018','2019','2020','2021']

prefix_url = "https://www.iplt20.com/stats/"
suffix_url = "/mostWkts?stats_type=bowling"

base_fname = 'iplstats'

In [79]:
for year in years:
    url = prefix_url + year + suffix_url
    page = requests.get(url, headers = headers)
    
    filename = base_fname + year + '_bowling.html'
    
    saveString(page.content, filename)
    time.sleep(5)

In [89]:
for year in years:
    
    filename = base_fname + year + '_bowling.html' 
    page = loadString(filename)

    soup = BeautifulSoup(page, 'html.parser')
    
    table = soup.find_all("table", class_ ="np-mostruns_table")[1]
    rows = table.find_all('tr')

    for i, row in enumerate(rows[1:]):
        stats = dict()
        data = row.find_all('td')
        
        if i == 0:
            player_name = data[1].find('h3').text
        else:
            player_name = data[1].find('h3', class_ ="np-team-name").text
            
        stats['matches'] = data[2].text
        stats['overs'] = data[4].text
        stats['runs'] = data[5].text
        stats['wickets'] = data[6].text
        stats['average'] = data[8].text
        stats['economy'] = data[9].text
        stats['strike_rate'] = data[10].text
        stats['4w'] = data[11].text
        stats['5w'] = data[12].text
        
        year_stats = {year : stats}
        
        if player_name not in player_dict.keys():
            player_dict[player_name] = dict()
            player_dict[player_name]['Bowling'] = [year_stats]
        elif 'Bowling' not in player_dict[player_name].keys():
            player_dict[player_name]['Bowling'] = [year_stats]
        else:
            player_dict[player_name]['Bowling'].append(year_stats)

In [90]:
player_dict

{'Kane Williamson': {'Batting': [{'2018': {'matches': '17',
     'not_outs': '3',
     'high_score': '735',
     'avg': '84',
     'Average': '52.50',
     'strike_rate': '142.44',
     'num_100': '0',
     'num_50': '8',
     'num_fours': '64',
     'num_six': '28'}},
   {'2019': {'matches': '9',
     'not_outs': '2',
     'high_score': '156',
     'avg': '70*',
     'Average': '22.28',
     'strike_rate': '120.00',
     'num_100': '0',
     'num_50': '1',
     'num_fours': '12',
     'num_six': '5'}},
   {'2020': {'matches': '12',
     'not_outs': '4',
     'high_score': '317',
     'avg': '67',
     'Average': '45.28',
     'strike_rate': '133.75',
     'num_100': '0',
     'num_50': '3',
     'num_fours': '26',
     'num_six': '10'}},
   {'2021': {'matches': '10',
     'not_outs': '4',
     'high_score': '266',
     'avg': '66*',
     'Average': '44.33',
     'strike_rate': '113.19',
     'num_100': '0',
     'num_50': '2',
     'num_fours': '29',
     'num_six': '2'}}],
  'Bowling

In [92]:
with open('Player_Data.json', 'w') as outfile:
    json.dump(player_dict, outfile)

In [93]:
player_list = [{key:player_dict[key]} for key in player_dict.keys()]

In [94]:
client = MongoClient('localhost', 27017)
ipl = client["IPL_data"]
player_stats = ipl["stats"]

In [95]:
player_stats.insert_many(player_list)

In [38]:
year_page = ['2018','2019','2020','2021']
prefix = "https://www.cricbuzz.com/cricket-series/ipl-" 
suffix = "/auction/completed"

# Headers to pass with the GET request
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"}
for page in year_page:
    # Modifying the search url
    search_url = prefix + page + suffix
    search_response = requests.get(search_url, headers = headers)
    search_content = str(search_response.content)
    time.sleep(5)
    filename = "iplauction" + page + '.html'
    saveString(search_content, filename)    

In [96]:
auction_dict = dict()
for page in year_page:

    filename = "iplauction" + page + '.html'
    iplauction = loadString(filename)

    soup = BeautifulSoup(iplauction, 'html.parser')
    boxes = soup.find_all('a', class_ = 'cb-col cb-col-100 cb-font-14 text-center')
    
    for box in boxes:
        data = dict()
        player_name = box.find('div',class_ = 'cb-font-18').text
        player_cat = box.find('div',class_ = 'cb-font-12 text-gray').text

        info = box.find_all('div', class_ = 'cb-font-16')
        for datapoint in info:
            presib = datapoint.find_previous_sibling('div').text
            if presib == 'Base Price':
                base_price = datapoint.text.replace('\xa0',' ')
            elif presib == 'Final Price':
                final_price = datapoint.text.replace('\xa0',' ')
            else :
                sold_to = datapoint.text

        data['player_cat'] = player_cat
        data['base_price'] = base_price
        data['final_price'] = final_price
        data['sold_to'] = sold_to

        year_data = {page: data}
        
        if player_name not in auction_dict.keys():
            auction_dict[player_name] = [year_data]
        else:
            auction_dict[player_name].append(year_data)

In [97]:
auction_list = [{key:auction_dict[key]} for key in auction_dict.keys()]

In [98]:
auction_list

[{' T N Dhillon ': [{'2018': {'player_cat': ' India ',
     'base_price': '20.00  Lakh',
     'final_price': '55.00  Lakh',
     'sold_to': 'MI'}}]},
 {' Javon Searles ': [{'2018': {'player_cat': ' Bowling Allrounder • Windies ',
     'base_price': '30.00  Lakh',
     'final_price': '30.00  Lakh',
     'sold_to': 'KKR'}}]},
 {' Manzoor Dar ': [{'2018': {'player_cat': ' Batsman • India ',
     'base_price': '20.00  Lakh',
     'final_price': '20.00  Lakh',
     'sold_to': 'PBKS'}}]},
 {' Jack Wildermuth ': [{'2018': {'player_cat': ' Bowler • Australia ',
     'base_price': '20.00  Lakh',
     'final_price': '-',
     'sold_to': ''}},
   {'2021': {'player_cat': ' Bowling Allrounder • Australia ',
     'base_price': '50.00  Lakh',
     'final_price': '-',
     'sold_to': ''}}]},
 {' Baba Indrajith ': [{'2018': {'player_cat': ' Batsman • India ',
     'base_price': '20.00  Lakh',
     'final_price': '-',
     'sold_to': ''}},
   {'2019': {'player_cat': ' Batsman • India ',
     'base_price

In [99]:
with open('Auction_Data.json', 'w') as outfile:
    json.dump(auction_dict, outfile)

In [100]:
player_auction = ipl["auction_info"]
player_auction.insert_many(auction_list)